In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
sample_query = tf.constant(np.random.rand(100, 5, 30).astype("float32"))# 30次元、10配列のデータが100個あるという想定
sample_key   = tf.constant(np.random.rand(100, 10, 40).astype("float32"))# 20次元、10配列のデータが100個あるという想定
sample_value = tf.constant(np.random.rand(100, 10, 40).astype("float32"))# 20次元、10配列のデータが100個あるという想定

# Multihead イメージ

In [ ]:
sample_ = tf.constant([
    [1 ,2 ,3 ,4 ],
    [4 ,5 ,6 ,7 ],
    [7 ,8 ,9 ,10],
    [10,11,12,13],
    [13,14,15,16],
    [16,17,18,19],
  ]
    
)

In [ ]:
tf.reshape(sample_, [3,2,4])

<tf.Tensor: shape=(3, 2, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 4,  5,  6,  7]],

       [[ 7,  8,  9, 10],
        [10, 11, 12, 13]],

       [[13, 14, 15, 16],
        [16, 17, 18, 19]]], dtype=int32)>

# Multiheadの実験

In [ ]:
hidden_dim = 40
head_num = 2

In [ ]:
 q_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='q_dense_layer')
 k_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='k_dense_layer')
 v_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='v_dense_layer')

In [ ]:
q = q_dense_layer(sample_query)
k = k_dense_layer(sample_key  )
v = v_dense_layer(sample_value)

In [ ]:
batch_num, seq_length, enb_dim = tf.unstack(q.shape)

In [ ]:
emb_dim_after_split = tf.cast((enb_dim / head_num), tf.int32)

In [ ]:
def split_head_dev(tensor_, head_num):
  batch_num, seq_length, enb_dim = tf.unstack(tensor_.shape)
  emb_dim_after_split = tf.cast((enb_dim / head_num), tf.int32)

  tensor_ = tf.transpose(tensor_, perm = [0,2,1])
  tensor_ = tf.reshape(tensor_, [batch_num.numpy(), head_num, emb_dim_after_split.numpy() ,seq_length.numpy() ])
  return tensor_ # (batch数, head番号, 埋め込み次元, seqの長さ)

In [ ]:
def unite_head_dev(tensor_,head_num):
    batch_num, head_num, emb_dim_after_split, seq_length  = tf.unstack(tensor_.shape)
    tensor_ = tf.reshape(tensor_, [batch_num, head_num * emb_dim_after_split, seq_length])
    return tf.transpose(tensor_, perm = [0,2,1])

In [ ]:
q_t = split_head_dev(q, head_num)
k_t = split_head_dev(k, head_num)
v_t = split_head_dev(v, head_num)

In [ ]:
weight= tf.matmul(q_t, k_t, transpose_a = True) #(batch数, head番号, queryのseq数, keyのseq数)

In [ ]:
weight.shape

TensorShape([100, 2, 5, 10])

In [ ]:
v_t.shape

TensorShape([100, 2, 20, 10])

In [ ]:
weighted = tf.matmul(weight, v_t,  transpose_b = True)

In [ ]:
weighted

<tf.Tensor: shape=(100, 2, 5, 20), dtype=float32, numpy=
array([[[[  9.770538  ,  -3.5800095 ,  -2.1258507 , ...,   7.591917  ,
           -0.75164294,  15.749116  ],
         [ 10.106265  ,  -3.5563834 ,  -2.284637  , ...,   7.391276  ,
           -1.3087375 ,  16.17132   ],
         [  2.1642168 ,  -0.7663706 ,  -1.3313485 , ...,   1.081059  ,
           -0.85595787,   3.971943  ],
         [ 10.933224  ,  -3.9258358 ,  -2.319636  , ...,   8.9402895 ,
           -0.49945652,  17.204584  ],
         [  7.4112034 ,  -2.6208947 ,  -1.8119187 , ...,   5.8800883 ,
           -0.3742365 ,  12.020581  ]],

        [[  1.313606  ,   0.3150173 ,  -1.6817917 , ...,  -1.3453654 ,
           -0.72953117,  -1.2131183 ],
         [ -1.9180024 ,  -0.10385267,   3.4488947 , ...,   1.8027523 ,
            0.556754  ,   1.9358432 ],
         [  1.422888  ,   0.08286929,  -1.3156557 , ...,  -1.8121426 ,
           -1.0511608 ,  -1.3319492 ],
         [ -3.818559  ,   0.10660833,   6.007265  , ...,   2.

In [ ]:
unite_head_dev(weighted, head_num)

<tf.Tensor: shape=(100, 5, 40), dtype=float32, numpy=
array([[[  9.770539  ,  -3.5800092 ,  -2.1258504 , ...,  -1.3453654 ,
          -0.72953117,  -1.2131183 ],
        [ 10.106265  ,  -3.5563834 ,  -2.284637  , ...,   1.8027523 ,
           0.556754  ,   1.9358432 ],
        [  2.1642168 ,  -0.76637065,  -1.3313485 , ...,  -1.8121426 ,
          -1.0511608 ,  -1.3319492 ],
        [ 10.933224  ,  -3.9258356 ,  -2.3196359 , ...,   2.95513   ,
           1.4284697 ,   3.1551723 ],
        [  7.4112034 ,  -2.6208947 ,  -1.8119187 , ...,  -1.1558592 ,
          -0.44018915,  -0.8649281 ]],

       [[  7.428206  ,  -2.9152408 ,  -0.04073229, ...,   1.2674097 ,
          -0.42886236,   0.66200274],
        [  6.2751665 ,  -2.3684757 ,  -0.45523453, ...,  -0.30082458,
          -0.71086425,  -1.1372238 ],
        [ -1.844934  ,   0.33768263,  -0.31626213, ...,  -5.2715816 ,
          -4.2512627 ,  -7.874365  ],
        [  7.800476  ,  -2.8890824 ,  -0.34378242, ...,  -0.39803684,
          

# multihead att

In [40]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, hidden_dim, head_num, dropout_rate):
    super().__init__()
    #self.hidden_dim   = hidden_dim
    self.head_num     = head_num
    self.dropout_rate = dropout_rate

    self.q_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='q_dense_layer' ,activation = "relu")
    self.k_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='k_dense_layer' ,activation = "relu")
    self.v_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='v_dense_layer' ,activation = "relu")
    self.output_dense_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False, name='output_dense_layer')
    self.attention_dropout_layer = tf.keras.layers.Dropout(dropout_rate)

  def split_head(self, tensor_):
    # split後の情報をまとめる。
    batch_num, seq_length, enb_dim = tf.unstack(tensor_.shape)

    emb_dim_after_split = tf.cast((enb_dim / self.head_num), tf.int32)
    tensor_ = tf.transpose(tensor_, perm = [0,2,1])

    return tf.reshape(tensor_, [batch_num.numpy(), self.head_num, emb_dim_after_split.numpy(), seq_length.numpy()]), emb_dim_after_split.numpy()

  def unite_head(self, tensor_):
    batch_num, head_num, emb_dim_after_split, seq_length = tf.unstack(tensor_.shape)
    tensor_ =  tf.reshape(tensor_, [batch_num, head_num * emb_dim_after_split, seq_length])

    return tf.transpose(tensor_, perm = [0,2,1])

  def call(self, key, query, value):

    key   = self.k_dense_layer(key  ) #(batch_size, seq_length, emb_dim)
    query = self.q_dense_layer(query) #(batch_size, seq_length, emb_dim)
    value = self.v_dense_layer(value) #(batch_size, seq_length, emb_dim)

    key   , emb_dim_key   = self.split_head(key)
    query , emb_dim_query = self.split_head(query)
    value , emb_dim_value = self.split_head(value)


    att_weight = tf.nn.softmax(tf.matmul(query, key, transpose_a = True) / emb_dim_key**0.5 )
    att_weight = self.attention_dropout_layer(att_weight)

    weighted= tf.matmul(value, att_weight)

    # add / norm
    return self.output_dense_layer(self.unite_head(weighted))
    #return key, att_weight,value 
    

In [41]:
att = MultiHeadAttention(40, 5, 0.9)

In [42]:
res = att(sample_key,sample_query, sample_query)

In [ ]:
res

# Set transformer

In [92]:
"""
MAB(X, Y ) = LayerNorm(H + rFF(H)), (6)
where H = LayerNorm(X + Multihead(X, Y, Y ; ω)),(7)
"""
class MAB(tf.keras.layers.Layer):
  def __init__(self, hidden_dim, head_num, dropout_rate):
    super().__init__()
    self.hidden_dim   = hidden_dim
    self.head_num     = head_num
    self.dropout_rate = dropout_rate


    self.att = MultiHeadAttention( hidden_dim, head_num, dropout_rate)
    self.dense_1 = tf.keras.layers.Dense(hidden_dim, use_bias=False,activation = "relu")
    self.dense_2 = tf.keras.layers.Dense(hidden_dim, use_bias=False,activation = "relu")

    self.norm    = tf.keras.layers.Normalization(-1)

  def call(self, x, y):
    
    mab_xy = self.att(x, y, y)
    h = self.norm(mab_xy + x)

    h_rff = self.dense_2(self.dense_1( h ))


    return self.norm(h + h_rff)



In [93]:
mab = MAB(40, 5, 0.9)

In [ ]:
mab(sample_key,sample_query)

In [95]:
# SAB(X) := MAB(X, X).
class SAB(tf.keras.layers.Layer):
  def __init__(self, hidden_dim, head_num, dropout_rate):
    super().__init__()
    self.hidden_dim   = hidden_dim
    self.head_num     = head_num
    self.dropout_rate = dropout_rate

    self.mab = MAB(hidden_dim, head_num, dropout_rate)

  
  def call(self, x):  


    return self.mab(x, x)




In [96]:
sab = SAB(40, 5, 0.9)

In [153]:
#PMAk(Z) = MAB(S,rFF(Z)) ∈ R
# k = 1で実装
class PMA(tf.keras.layers.Layer):
  def __init__(self, hidden_dim, head_num, dropout_rate, k):
    super().__init__()
    #self.batch_size   = batch_size
    self.hidden_dim   = hidden_dim
    self.head_num     = head_num
    self.dropout_rate = dropout_rate
    self.k = k
    
    self.dense_1 = tf.keras.layers.Dense(hidden_dim, use_bias=False,activation= "relu")
    self.dense_2 = tf.keras.layers.Dense(hidden_dim, use_bias=False,activation= "relu")
    self.dense_s = tf.keras.layers.Dense(hidden_dim, use_bias=False,activation= "relu")

    self.mab = MAB(hidden_dim, head_num, dropout_rate)
    self.seed_vectors = tf.random.uniform(shape=(1, k, hidden_dim), minval = 0.0, maxval = 1.0)

  def call(self, input):  
    #s = tf.repeat(self.seed_vectors , repeats = int( input.shape[1] / self.k ), axis = 1)
    
    batch_num, seq_length, enb_dim = tf.unstack(input.shape)
    s = tf.repeat(self.seed_vectors , repeats = batch_num, axis = 0)
    
    
    rff_z = self.dense_2(self.dense_1(input))
    s = self.dense_s(self.seed_vectors)

    res = self.mab(s, rff_z)
    return tf.reshape(res, [batch_num, self.hidden_dim])
    #return self.seed_vectors, rff_z

In [154]:
pma = PMA(40, 8, 0.9, 1)

In [99]:
res = pma(sample_key)

In [ ]:
res

In [222]:
class SetTransformer(tf.keras.Model):
  def __init__(self, hidden_dim, head_num, dropout_rate, k, output_shape):
    super().__init__()
    #self.batch_size   = batch_size
    self.hidden_dim   = hidden_dim
    self.head_num     = head_num
    self.dropout_rate = dropout_rate
    self.k = k
    
    self.sab_1 = SAB(hidden_dim, head_num, dropout_rate)
    self.sab_2 = SAB(hidden_dim, head_num, dropout_rate)

    self.pma = PMA(hidden_dim, head_num, dropout_rate, k)

    self.dense_1 = tf.keras.layers.Dense(output_shape, use_bias=False )

  def call(self, input): 
    enc_output = self.sab_2(self.sab_1(input))

    #return self.pma(enc_output)
    return self.dense_1(self.pma(enc_output))

In [223]:
stf = SetTransformer(40, 8, 0.9, 1, 1)

# 実験

20次元のベクトルのノルムを求める

In [224]:
data = np.random.rand(10000, 10,10)

In [225]:
max_ = np.linalg.norm( np.array([10,10,10,10,10,10,10,10,10,10 ]) )

In [226]:
ans = np.linalg.norm( np.sum(data, axis = 1),axis = 1) / max_

In [227]:
dataset = tf.data.Dataset.from_tensor_slices({"data":data, "ans":ans} )

In [228]:
model = SetTransformer(10, 2, 0.9, 1, 1)

In [229]:
opt = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)
mae = tf.keras.losses.MeanAbsoluteError()
for epoch in range(30):
  total_loss = 0
  cnt = 0
  # Training loop - using batches of 32
  for num, d in enumerate(dataset.batch(10)):
    with tf.GradientTape() as tape:
      outputs = model(d["data"])
      loss = tf.reduce_mean( mae(d["ans"], outputs) )
    gradients = tape.gradient(loss, model.trainable_variables)

    opt.apply_gradients(zip(gradients, model.trainable_variables))
    total_loss += loss
    cnt += 1
  print("epoch = {}, num = {},loss = {}".format(epoch, num, total_loss / cnt))

epoch = 0, num = 999,loss = 0.06303281337022781
epoch = 1, num = 999,loss = 0.03308391198515892
epoch = 2, num = 999,loss = 0.02248024381697178
epoch = 3, num = 999,loss = 0.020442595705389977
epoch = 4, num = 999,loss = 0.019643640145659447
epoch = 5, num = 999,loss = 0.01943448930978775
epoch = 6, num = 999,loss = 0.01924673467874527


KeyboardInterrupt: ignored

In [230]:
test_data = np.random.rand(1, 10,10)

In [231]:
test_data_shuffle = np.concatenate(  [test_data[0, 5:], test_data[0, 0:5]]).reshape(1,10,10)

In [232]:
model(test_data)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.5137366]], dtype=float32)>

In [233]:
model(test_data_shuffle)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.51373667]], dtype=float32)>

In [234]:
np.linalg.norm( np.sum(test_data, axis = 1),axis = 1) / max_

array([0.53719827])